In [4]:
import requests
import json
import datetime
from kafka import KafkaProducer
import uuid

# Define your API key and coordinates
api_key = 'cd29aaa47fb913ae2780eb4f2c8461ca'
lat = '10.762622'
lon = '106.660172'
topic_name='WeatherData',
kafka_producer = KafkaProducer(bootstrap_servers=['kafka:9093'], api_version=(0, 10))


date = datetime.datetime.now()
time_now = int(date.strftime("%H")) + 7
time_expr = 'sang' if 6 <= time_now <= 15 else 'toi'

month_now = int(date.strftime("%m"))
month_expr = 'mua' if 5 <= month_now <= 11 else 'kho'

time_label = 0 if time_expr == 'sang' else 1
month_label = 0 if month_expr == 'mua' else 1
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric'

# Make a GET request to the API
response = requests.get(url)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    weather_data = response.json()
    kafka_producer = KafkaProducer(bootstrap_servers=['kafka:9093'], api_version=(0, 10))
        # Access and print specific weather information
    city_name = weather_data['name']
    temperature = weather_data['main']['temp']
    feelslike = weather_data['main']['feels_like']
    wind = weather_data['wind']['speed']
    cloud = weather_data['clouds']['all']
    humidity = weather_data['main']['humidity']
    pressure = weather_data['main']['pressure']
    weather = weather_data['weather'][0]['description']
    print(f'City: {city_name}')
    print(f'Temperature: {temperature}°C')
    print(f'Feels Like: {feelslike}°C')
    print(f'Wind Degree: {wind}°')
    print(f'Cloud Coverage: {cloud}%')
    print(f'Humidity: {humidity}%')
    print(f'Pressure: {pressure} hPa')
    print(f'time: {time_expr}')
    print(f'month: {month_expr}')
    print(f'weather: {weather}')

    data = {
            'temperature': temperature,
            'feelslike': feelslike,
            'wind': wind,
            'cloud': cloud,
            'humidity': humidity,
            'time': time_label,
            'month': month_label,
            'pressure': pressure,
            'weather': weather
        }
    
    key = str(uuid.uuid4())
    value = json.dumps(data)
    print(key)
    print(value)
    key_bytes = bytes(key, encoding='utf-8')
    value_bytes = bytes(value, encoding='utf-8')
    kafka_producer.send('WeatherData', key=key_bytes, value=value_bytes)
    kafka_producer.flush()
    print('Message published successfully.')
   
    #kafka_producer.close()

else:
    print(f'Error: Unable to fetch weather data. Status code: {response.status_code}')



City: Ho Chi Minh City
Temperature: 25.99°C
Feels Like: 25.99°C
Wind Degree: 1.54°
Cloud Coverage: 40%
Humidity: 100%
Pressure: 1011 hPa
time: toi
month: mua
42d3c65c-40da-4693-8e0e-86c6b6c307be
{"temperature": 25.99, "feelslike": 25.99, "wind": 1.54, "cloud": 40, "humidity": 100, "pressure": 1011, "time": "toi", "month": "mua"}
Message published successfully.


In [ ]:
import requests
import json
import datetime
from kafka import KafkaProducer
import uuid
import time
import schedule

def fetch_and_publish_weather():
    # Define your API key and coordinates
    api_key = 'cd29aaa47fb913ae2780eb4f2c8461ca'
    lat = '10.762622'
    lon = '106.660172'

    kafka_producer = KafkaProducer(bootstrap_servers=['kafka:9093'], api_version=(0, 10))

    date = datetime.datetime.now()
    time_now = int(date.strftime("%H")) + 7
    time_expr = 'sang' if 6 <= time_now <= 15 else 'toi'

    month_now = int(date.strftime("%m"))
    month_expr = 'mua' if 5 <= month_now <= 11 else 'kho'

    time_label = 0 if time_expr == 'sang' else 1
    month_label = 0 if month_expr == 'mua' else 1
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric'

    # Make a GET request to the API
    response = requests.get(url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        weather_data = response.json()

        # Access and print specific weather information
        city_name = weather_data['name']
        temperature = weather_data['main']['temp']
        feelslike = weather_data['main']['feels_like']
        wind = weather_data['wind']['speed']
        cloud = weather_data['clouds']['all']
        humidity = weather_data['main']['humidity']
        pressure = weather_data['main']['pressure']
        weather = weather_data['weather']['description']


        print(f'City: {city_name}')
        print(f'Temperature: {temperature}°C')
        print(f'Feels Like: {feelslike}°C')
        print(f'Wind Degree: {wind}°')
        print(f'Cloud Coverage: {cloud}%')
        print(f'Humidity: {humidity}%')
        print(f'Pressure: {pressure} hPa')
        print(f'time: {time_expr}')
        print(f'month: {month_expr}')
        print(f'weather: {weather}')

        data = {
            'temperature': temperature,
            'feelslike': feelslike,
            'wind': wind,
            'cloud': cloud,
            'humidity': humidity,
            'time': time_label,
            'month': month_label,
            'pressure': pressure,
            'weather': weather
        }

        key = str(uuid.uuid4())
        value = json.dumps(data)
        print(key)
        print(value)
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        kafka_producer.send('WeatherData', key=key_bytes, value=value_bytes)
        kafka_producer.flush()
        print('Message published successfully.')
    else:
        print(f'Error: Unable to fetch weather data. Status code: {response.status_code}')

# Lên lịch công việc mỗi phút
schedule.every(15).seconds.do(fetch_and_publish_weather)

while True:
    schedule.run_pending()
    time.sleep(1)
